<a href="https://colab.research.google.com/github/c-roets/delayNN/blob/main/DelayNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Smart Instrumentation Delay between peaks - Neural Network (Calvin Roets)

In [135]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math
import os
import tensorflow as tf
import pandas as pd
import natsort

from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from zipfile import ZipFile

In [136]:
standaarddirectory = os.getcwd()
file_name = "delay_dataset.zip"
if not os.path.exists("stud_train"):
  with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print("extractie data klaar")

if not os.path.exists("stud_train_combinedASDR"):
  os.makedirs("stud_train_combinedASDR")



In [137]:
# constants
train_set = 'stud_train'
test_set = 'stud_test'
i = 0
nummer = 1
evencheck = 0
x = np.arange(0, 1024)
k = 0

In [138]:
#Geeft een array met strings weer waarin de namen van de files in die map staan
def laadarraymetfilenamenuitmap(map):
    j = 0
    filenames = [None] * 600  # array met alle filenames van in folder
    # iterate over files in that directory
    for filename in os.listdir(train_set):
        f = os.path.join(train_set, filename)
        # checking if it is a file
        if os.path.isfile(f):
            filenames[j] = f
            j += 1
    filenames.sort()
    return filenames

In [139]:
#plot het signaal van een numpy array aan de hand van een megegeven signaalbestandsnaam
def plotsignaal(signaal): #signaal is naam bestand tussen aanhalingstekens
    if evencheck % 2 == 0:
        kleur = "red"
    else:
        kleur = "blue"
    plot = plt.figure(nummer)
    plt.xlabel("X axis")
    plt.ylabel("Y axis")
    plt.plot(x,np.load(signaal), color=kleur)

In [140]:
#Geeft de waarde van de positie van die piek terug aan de hand van de bestandsnaam
def geefpositiepiek(signaal):
  truncatedsignaal = (signaal.split("P_",1)[1]) #alles voor de xxx wegdoen dus BV ASDR_P_ wegdoen
  piek = ''.join(x for x in truncatedsignaal[0:3] if x.isdigit()) 
  return piek


In [141]:
#voegt de array van twee signalen samen om 1 vector van 1024 samples te bekomen
def voegsamen(signaal1, signaal2):
  return np.concatenate([np.load(signaal1), np.load(signaal2)])

In [142]:
#main
filenames = laadarraymetfilenamenuitmap(train_set)

filenamesASDR = filenames[0:200]
filenamesGAUSS = filenames[200:400]
filenamesSBOX = filenames[400:600]
trainingdata = np.empty   #array met arrays van 1024 samples
delaypiek = []    #array met actuele delays tussen de pieken (validatiecijfers voor training)

for data in filenamesASDR:
  evencheck += 1
  if k != 0 and evencheck % 2 == 0:
    masterfile = voegsamen(previous, data)            #MASTERFILE IS DE ARRAY VAN 1024 SAMPLES VAN DE TWEE SIGNALEN SAMEN
    os.chdir("stud_train_combinedASDR")
    savename = f"{nummer}.npy"
    np.save(savename,masterfile)
    os.chdir(standaarddirectory)
    #trainingdata = np.append((trainingdata, masterfile)) #array in array maar werkt niet, voegt alle arrays na elkaar in plaats van in elkaar
    nummer += 1
    delaypiek.append(512-int(geefpositiepiek(previous))+int(geefpositiepiek(data)))

    #plotten voor eerste paar grafieken, mooie visualisatie als controle
    if evencheck < 1:  #evencheck > aantal grafieken*2, max 41 vanwege memory limiet
      plot= plt.figure(nummer)
      plt.plot(x,masterfile, color="red")

  k += 1
  previous = data

ASDRCOMBINEDFILES = os.listdir("stud_train_combinedASDR")
natsort.natsorted(ASDRCOMBINEDFILES)

['1.npy',
 '2.npy',
 '3.npy',
 '4.npy',
 '5.npy',
 '6.npy',
 '7.npy',
 '8.npy',
 '9.npy',
 '10.npy',
 '11.npy',
 '12.npy',
 '13.npy',
 '14.npy',
 '15.npy',
 '16.npy',
 '17.npy',
 '18.npy',
 '19.npy',
 '20.npy',
 '21.npy',
 '22.npy',
 '23.npy',
 '24.npy',
 '25.npy',
 '26.npy',
 '27.npy',
 '28.npy',
 '29.npy',
 '30.npy',
 '31.npy',
 '32.npy',
 '33.npy',
 '34.npy',
 '35.npy',
 '36.npy',
 '37.npy',
 '38.npy',
 '39.npy',
 '40.npy',
 '41.npy',
 '42.npy',
 '43.npy',
 '44.npy',
 '45.npy',
 '46.npy',
 '47.npy',
 '48.npy',
 '49.npy',
 '50.npy',
 '51.npy',
 '52.npy',
 '53.npy',
 '54.npy',
 '55.npy',
 '56.npy',
 '57.npy',
 '58.npy',
 '59.npy',
 '60.npy',
 '61.npy',
 '62.npy',
 '63.npy',
 '64.npy',
 '65.npy',
 '66.npy',
 '67.npy',
 '68.npy',
 '69.npy',
 '70.npy',
 '71.npy',
 '72.npy',
 '73.npy',
 '74.npy',
 '75.npy',
 '76.npy',
 '77.npy',
 '78.npy',
 '79.npy',
 '80.npy',
 '81.npy',
 '82.npy',
 '83.npy',
 '84.npy',
 '85.npy',
 '86.npy',
 '87.npy',
 '88.npy',
 '89.npy',
 '90.npy',
 '91.npy',
 '92.npy

In [143]:
print(filenamesASDR)
print(len(filenamesASDR))
print(delaypiek)
print(len(delaypiek))

['stud_train/ADSR0_P_282_SNR20DB.npy', 'stud_train/ADSR0_P_343_SNR20DB.npy', 'stud_train/ADSR10_P_184_SNR10DB.npy', 'stud_train/ADSR10_P_277_SNR10DB.npy', 'stud_train/ADSR11_P_101_SNR11DB.npy', 'stud_train/ADSR11_P_154_SNR11DB.npy', 'stud_train/ADSR12_P_226_SNR10DB.npy', 'stud_train/ADSR12_P_265_SNR10DB.npy', 'stud_train/ADSR13_P_279_SNR16DB.npy', 'stud_train/ADSR13_P_333_SNR16DB.npy', 'stud_train/ADSR14_P_231_SNR33DB.npy', 'stud_train/ADSR14_P_325_SNR33DB.npy', 'stud_train/ADSR15_P_188_SNR27DB.npy', 'stud_train/ADSR15_P_275_SNR27DB.npy', 'stud_train/ADSR16_P_104_SNR11DB.npy', 'stud_train/ADSR16_P_164_SNR11DB.npy', 'stud_train/ADSR17_P_257_SNR18DB.npy', 'stud_train/ADSR17_P_322_SNR18DB.npy', 'stud_train/ADSR18_P_215_SNR10DB.npy', 'stud_train/ADSR18_P_293_SNR10DB.npy', 'stud_train/ADSR19_P_202_SNR27DB.npy', 'stud_train/ADSR19_P_260_SNR27DB.npy', 'stud_train/ADSR1_P_295_SNR12DB.npy', 'stud_train/ADSR1_P_341_SNR12DB.npy', 'stud_train/ADSR20_P_271_SNR16DB.npy', 'stud_train/ADSR20_P_310_SNR

In [144]:
os.chdir(standaarddirectory)
os.chdir("stud_train_combinedASDR")
#dfASDR = pd.DataFrame(data = np.load("1.npy"))
#dfASDR = pd.DataFrame(data = np.load("2.npy"))

df = pd.DataFrame(np.load(f"{x}.npy") for x in range(1, 101))
#for combinedfile in ASDRCOMBINEDFILES:

#df = df.T Transponeren van het dataframe
#print(dfASDR)
print(df)
os.chdir(standaarddirectory)

        0         1         2         3         4         5         6     \
0   0.005852 -0.019865 -0.042075 -0.041071 -0.018101 -0.027462  0.036881   
1   0.056715  0.132339  0.135539  0.049227  0.065208 -0.077982  0.018259   
2   0.027773  0.005639 -0.002636 -0.167840  0.038056  0.023487  0.007777   
3  -0.026466  0.008621 -0.161340 -0.012341 -0.080992 -0.166868 -0.102965   
4   0.013772 -0.008582 -0.079611 -0.002194 -0.036131 -0.035800  0.019848   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.151431 -0.043660  0.028297  0.002026 -0.098621  0.052707 -0.026667   
96  0.005700 -0.042164  0.003313 -0.028427 -0.041338  0.066976  0.014467   
97  0.014818  0.002778  0.014760 -0.012460 -0.015580 -0.018382  0.006778   
98 -0.006699  0.019235 -0.052493 -0.053116  0.046434  0.116708 -0.173489   
99 -0.005460 -0.031761 -0.049384  0.026614  0.045476 -0.008789  0.021138   

        7         8         9     ...      1014      1015      1016      1017  \
0  -0.

In [145]:
dfdelay = pd.DataFrame(data = delaypiek)
print(dfdelay)

      0
0   573
1   605
2   565
3   551
4   566
..  ...
95  564
96  567
97  575
98  564
99  578

[100 rows x 1 columns]


In [146]:
# Create a model
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

In [147]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                32        
                                                                 
 dense_31 (Dense)            (None, 16)                272       
                                                                 
 dense_32 (Dense)            (None, 16)                272       
                                                                 
 dense_33 (Dense)            (None, 16)                272       
                                                                 
 dense_34 (Dense)            (None, 1)                 17        
                                                                 
Total params: 865
Trainable params: 865
Non-trainable params: 0
_________________________________________________________________


In [148]:
# Add optimizer, loss function, and metrics to model and compile it
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

In [149]:
# Train model
history = model.fit(df.values,
                    epochs=500,
                    batch_size=64,
                    validation_data=(dfdelay.values))

ValueError: ignored

In [ ]:
#reset